<a href="https://colab.research.google.com/github/Aliahmadjangohar/Aliahmadjangohar/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Set Up Google Colab**:

Open Google Colab (https://colab.research.google.com/).
Connect to a GPU if available. Go to "Runtime" > "Change runtime type" and select GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Navigate to the Dataset Folder:**

Use the cd command to navigate to the folder where your annotated dataset is stored.

In [ ]:
%cd /content/drive/MyDrive/Directions


/content/drive/MyDrive/Directions


## **Data Preprocessing:**

Organize your dataset into folders named 'left', 'right', and 'straight'. Each of these folders should contain both the original images and their corresponding XML annotation files

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np

# Define a function to parse the XML annotation file
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    # Extract the relevant information, e.g., 'class' (turn type)
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        return class_name  # Assuming only one object per image

# Define a function to process original images and annotations
def process_data(dataset_dir, output_dir):
    for category in os.listdir(dataset_dir):
        category_dir = os.path.join(dataset_dir, category)
        output_category_dir = os.path.join(output_dir, category)

        if not os.path.exists(output_category_dir):
            os.makedirs(output_category_dir)

        for filename in os.listdir(category_dir):
            if filename.endswith('.jpeg'):
                image_path = os.path.join(category_dir, filename)
                annotation_path = os.path.splitext(image_path)[0] + ".xml"

                # Check if the corresponding XML annotation file exists
                if os.path.exists(annotation_path):
                    # Load the image
                    image = cv2.imread(image_path)
                    # Perform any necessary image processing here

                    # Parse the annotation to get the label
                    label = parse_annotation(annotation_path)

                    # Save the processed image with the same filename
                    processed_image_path = os.path.join(output_category_dir, filename)
                    cv2.imwrite(processed_image_path, image)

                    # You can also save the label or use it as needed
                    # For example, you can save labels in a separate text file

# Define the path to your dataset folder and the output folder
dataset_dir = '/content/drive/MyDrive/Directions'
output_dir = '/content/drive/MyDrive/dataset cricket'

# Process the data
process_data(dataset_dir, output_dir)




## **Data Loading**:

Load and preprocess your dataset. You can use libraries like OpenCV or PIL for image loading and parsing the XML files for annotations

In [ ]:
# Define a function to parse the XML annotation file
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    # Extract the relevant information, e.g., 'class' (turn type)
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        return class_name  # Assuming only one object per image





In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np

# Define a function to parse the XML annotation file
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    # Initialize the class name variable
    class_name = None

    # Iterate through objects in the XML and find the class name
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        break  # Assuming only one object per image, so break after finding one

    return class_name

# Define a function to load and preprocess the dataset
def load_and_preprocess_dataset(dataset_dir, image_extension='.jpg', annotation_extension='.xml'):
    images = []
    labels = []

    for filename in os.listdir(dataset_dir):
        if filename.endswith(image_extension):
            image_path = os.path.join(dataset_dir, filename)
            annotation_filename = filename.replace(image_extension, annotation_extension)
            annotation_path = os.path.join(dataset_dir, annotation_filename)

            # Check if the corresponding XML annotation file exists
            if os.path.exists(annotation_path):
                # Load the image without resizing
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Parse the annotation to get the label
                label = parse_annotation(annotation_path)

                if label is not None:
                    images.append(image)
                    labels.append(label)

    return np.array(images), np.array(labels)

# Define the path to your dataset folder
dataset_dir = '/content/drive/MyDrive/Directions'

# Specify the file extensions
image_extension = '.jpeg'
annotation_extension = '.xml'

# Load and preprocess the dataset
images, labels = load_and_preprocess_dataset(dataset_dir, image_extension, annotation_extension)

# Check the shape of the loaded data
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)






Images shape: (0,)
Labels shape: (0,)


##**Split the Dataset**:

Split your dataset into training and validation sets. Typically, you'd use around 70-80% of the data for training and the rest for validation. You can use the train_test_split function from scikit-learn to achieve this.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets (e.g., 80% training, 20% validation)
test_size = 0.2  # Adjust this percentage as needed
random_state = 42  # For reproducibility

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=random_state)

# Check the shapes of the training and validation sets
print("Training images shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validation images shape:", X_val.shape)
print("Validation labels shape:", y_val.shape)


##**Model Building**:

Build a convolutional neural network (CNN) for image classification. You can use popular deep learning frameworks like TensorFlow or PyTorch. Define the architecture of your CNN model.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional Layer 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional Layer 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional Layer 3 (optional)
    # You can add more convolutional layers as needed.

    # Flatten the feature maps
    model.add(layers.Flatten())

    # Fully Connected Layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Optional dropout layer for regularization
    model.add(layers.Dense(num_classes, activation='softmax'))  # Output layer

    return model

# Define the input shape and number of classes
input_shape = (your_image_height, your_image_width, your_num_channels)  # Replace with actual values
num_classes = your_num_classes  # Replace with the number of classes in your dataset

# Create the CNN model
model = create_cnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
              metrics=['accuracy'])

# Print the model summary
model.summary()


## Compile the Model:

Compile your model with an appropriate loss function (e.g., categorical cross-entropy) and optimizer (e.g., Adam

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
              metrics=['accuracy'])


## **Training**:

Train your model using the training dataset. Use the validation dataset to monitor the model's performance during training. Here's an example of how to train a model in TensorFlow

In [ ]:
# Specify the number of epochs and batch size for training
epochs = 10  # Adjust the number of epochs as needed
batch_size = 32  # Adjust the batch size as needed

# Train the model using the training and validation datasets
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Evaluate the model on the test dataset (if you have one)
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print("Test Loss:", test_loss)
# print("Test Accuracy:", test_accuracy)


## Model Evaluation:

Evaluate your trained model using appropriate metrics such as accuracy, precision, recall, and F1-score. Here's an example of how to calculate these metrics

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(val_generator)
y_true = val_generator.classes
class_names = list(val_generator.class_indices.keys())

report = classification_report(y_true, np.argmax(y_pred, axis=1), target_names=class_names)
print(report)


**Save the Model**:

Once you are satisfied with your model's performance, save it for future use

In [ ]:
model.save('road_classification_model.h5')
